In [2]:
import stltovoxel as stv
import numpy as np
import nrrd
from stl import mesh

# Load the STL file

In [59]:
def checkmeshparam(file_path, resolution = 100):
    mesh_obj = mesh.Mesh.from_file(file_path)
    org_mesh = np.hstack((mesh_obj.v0[:, np.newaxis], mesh_obj.v1[:, np.newaxis], mesh_obj.v2[:, np.newaxis]))
    obj = []
    obj.append(org_mesh)
    vol, scale, shift = stv.convert_meshes(obj, resolution, None, False)
    vol = vol.astype(np.uint8)
    return vol, vol.shape, scale, shift    

In [60]:
def convertmesh(filepath):
    vol, size, sfa, shift = checkmeshparam(file_path)
    sf = np.mean(sfa)
    if sf <= 0.975 or sf >= 1.025:
        resolution = int(np.floor(100/(np.mean(sf))))
        print(f"downscaled to {resolution}") 
        vol, size, sfa, shift = checkmeshparam(filepath, resolution)
        
    return vol, size, shift

In [65]:
def createnrrd(vol, size, shift):
    origin = (shift[0],shift[1],shift[2])
    trans_vol = np.rot90(np.flip(vol, axis = 2), k =1, axes = (0,2))
    header = {
    'type' : 'u1',
    'dimension' : '3',
    'space': 'left-posterior-superior',
    'sizes': size,
    'space directions': [(1, 0, 0), (0, 1, 0), (0, 0, 1)],
    'kinds': ['domain', 'domain', 'domain'],
    'encoding': 'gzip',
    'space origin': origin
    }
    nrrd.write('implant4_translated_uint8_h.nrrd', trans_vol, header) 
    print("implant4_translated_uint8_h.nrrd created")
    

In [61]:
vol, size, shift = convertmesh('.\Skulls\B0004\B0004_implant.stl')

Processing layer 0/101
Processing layer 1/101
Processing layer 2/101
Processing layer 3/101
Processing layer 4/101
Processing layer 5/101
Processing layer 6/101
Processing layer 7/101
Processing layer 8/101
Processing layer 9/101
Processing layer 10/101
Processing layer 11/101
Processing layer 12/101
Processing layer 13/101
Processing layer 14/101
Processing layer 15/101
Processing layer 16/101
Processing layer 17/101
Processing layer 18/101
Processing layer 19/101
Processing layer 20/101
Processing layer 21/101
Processing layer 22/101
Processing layer 23/101
Processing layer 24/101
Processing layer 25/101
Processing layer 26/101
Processing layer 27/101
Processing layer 28/101
Processing layer 29/101
Processing layer 30/101
Processing layer 31/101
Processing layer 32/101
Processing layer 33/101
Processing layer 34/101
Processing layer 35/101
Processing layer 36/101
Processing layer 37/101
Processing layer 38/101
Processing layer 39/101
Processing layer 40/101
Processing layer 41/101
Pr

In [66]:
createnrrd(vol, size,shift)

implant4_translated_uint8_h.nrrd created
